In [1]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

# Monte Carlo Simulation and Linear Algebra

**CS1302 Introduction to Computer Programming**
___

In [2]:
import math
import random
import numpy as np
import ipywidgets as widgets
%reload_ext divewidgets

## Monte Carlo simulation

**What is Monte Carlo simulation?**

 > The name Monte Carlo refers to the [Monte Carlo Casino in Monaco](https://en.wikipedia.org/wiki/Monte_Carlo_Casino) where Ulam's uncle would borrow money from relatives to gamble.

It would be nice to simulate the casino so Ulam's uncle did not need to borrow money to play in the casino.  
Actually...,
- Monte Carlo is the code name of the secret project for creating the [hydrogen bomb](https://en.wikipedia.org/wiki/Monte_Carlo_method). 
- [Ulam](https://en.wikipedia.org/wiki/Stanislaw_Ulam) worked with [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann) to program the first electronic computer ENIAC to simulate a computational model of a thermonuclear reaction.

(See also [The Beginning of the Monte Carlo Method](https://permalink.lanl.gov/object/tr?what=info:lanl-repo/lareport/LA-UR-88-9067) for a more detailed account.)

**How to compute the value of $\pi$**?

An application of Monte Carlo simulation is to approximate $\pi$ using 
the [Buffon's needle](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem).  
There is [a program](https://www.khanacademy.org/computer-programming/pi-by-buffons-needle/6695500989890560) written in javascript to do this.

The javascript program is long, so we will use an alternative simulation that is easier to understand/program.

If we uniformly randomly pick a point in a square. What is the chance the point is inside the inscribed circle, i.e., the biggest circle inside the square?

The chance is the circle's area divided by the square's area. Suppose the square has length $\ell$, then the chance is

$$ \frac{\pi (\ell /2)^2}{ (\ell)^2 } = \frac{\pi}4 $$
independent of the length $\ell$.

**Exercise** Complete the following function to return an approximation of $\pi$ as follows:
1. Simulate the random process of picking a point from a square repeatedly `n` times by  
  generating the $x$ and $y$ coordinates uniformly randomly from a unit interval $[0,1)$.
2. Compute the fraction of times the point is in the first quadrant of the inscribed circle, as shown in the figure below.
3. Return $4$ times the fraction as the approximation.
<!--
<p><a href="https://commons.wikimedia.org/wiki/File:Pi_30K.gif#/media/File:Pi_30K.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" alt="Pi 30K.gif"></a></p>
-->

In [6]:
%%timeit -n 1 -r 2
def approximate_pi(n):
    return sum(4 if math.dist((random.random(), random.random()), (0, 0)) <= 1 else 0 for _ in range(n)) / n
print(f"Approximate: {approximate_pi(int(1e7))}\nGround truth: {math.pi}")

Approximate: 3.1417512
Ground truth: 3.141592653589793
Approximate: 3.142694
Ground truth: 3.141592653589793
3.13 s ± 16.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


**How accurate is the approximation?**

The following uses a powerful library `numpy` for computing to return a [$95\%$-confidence interval](http://onlinestatbook.com/2/estimation/mean.html#:~:text=To%20compute%20the%2095%25%20confidence,be%20between%20the%20cutoff%20points.).

In [7]:
%%timeit -n 1 -r 1
import numpy as np


def np_approximate_pi(n):
    in_circle = (np.random.random((n, 2)) ** 2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n ** 0.5
    return np.array([mean - 2 * std, mean + 2 * std])


interval = np_approximate_pi(int(1e7))
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

95%-confidence interval: [3.14041893 3.14249627]
Estimate: 3.1415 ± 0.0010
Ground truth: 3.141592653589793
619 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


```{note}

The computation done using `numpy` is over $5$ times faster despite the additional computation of the standard deviation.
```

```{seealso}

There are faster methods to approximate $\pi$, such as the [Chudnovsky_algorithm](https://en.wikipedia.org/wiki/Chudnovsky_algorithm), but the Monte-Carlo method is helpful in complicated situations.   
E.g., see the Monte Carlo simulation of a [real-life situation](https://www.youtube.com/watch?v=-fCVxTTAtFQ) in playing basketball:
> "When down by three and left with only 30 seconds, is it better to attempt a hard 3-point shot or an easy 2-point shot and get another possession?"   --LeBron James
```

## Linear Algebra

**How to solve a linear equation?**

Given the following linear equation in variable $x$ with real-valued coefficient $a$ and $b$,

$$ a x = b,$$
what is the value of $x$ that satisfies the equation?

**Exercise** Complete the following function to return either the unique solution of $x$ or `None` if a unique solution does not exist.

In [8]:
def solve_linear_equation(a, b):
    return b / a if a else None


@widgets.interact(a=(0, 5, 1), b=(0, 5, 1))
def linear_equation_solver(a=2, b=3):
    print(
        f"""linear equation: {a}x = {b}
       solution: x = {solve_linear_equation(a,b)}"""
    )

interactive(children=(IntSlider(value=2, description='a', max=5), IntSlider(value=3, description='b', max=5), …

**How to solve multiple linear equations?**

In the general case, we have a system of $m$ linear equations and $n$ variables:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 + \dots + a_{0(n-1)} x_{n-1} &= b_0\\
a_{10} x_0 + a_{11} x_1 + \dots + a_{1(n-1)} x_{n-1} &= b_1\\
 & \vdots\\
a_{(m-1)0} x_0 + a_{(m-1)1} x_1 + \dots + a_{(m-1)(n-1)} x_{n-1} &= b_{m-1}\\
\end{aligned}
$$
where
- $x_j$ for $j\in \{0,\dots,n-1\}$ are the variables, and
- $a_{ij}$ and $b_j$ for $i\in \{0,\dots,m-1\}$ and $j\in \{0,\dots,n-1\}$ are the coefficients.

A fundamental problem in linear algebra is to compute the unique solution to the system if it exists.

We will consider the simpler 2-by-2 system with 2 variables and 2 equations:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1.
\end{aligned}
$$

To get an idea of the solution, suppose 

$$a_{00}=a_{11}=1, a_{01} = a_{10} = 0.$$
The system of equations becomes

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{x_0 +}  x_1 &= b_1,
\end{aligned}
$$
which gives the solution directly.

What about $a_{00}=a_{11}=2$ instead?

$$ \begin{aligned}
2x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1,
\end{aligned}$$

To obtain the solution, we divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2.
\end{aligned}
$$

What if $a_{01}=2$ instead?

$$ \begin{aligned}
2x_0 + 2x_1 &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$

The second equation gives the solution of $x_1$, and we can use the solution in the first equation to solve for $x_0$. More precisely:
- Subtract the second equation from the first one:

$$ \begin{aligned}
2x_0 \hphantom{+2x_1} &= b_0 - b_1\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$
- Divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0 - b_1}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2\\
\end{aligned}
$$

```{important}

The above operations are called *row operations* in linear algebra: each row is an equation.  
A system of linear equations can be solved by the linear operations of 
1. multiplying an equation by a constant, and
2. subtracting one equation from another.
```

How to write a program to solve a general 2-by-2 system? We will use the `numpy` library.

### Creating `numpy` arrays

**How to store the coefficients?**

In linear algebra, a system of equations such as

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1
\end{aligned}
$$
is written concisely in *matrix* form as $ \mathbf{A} \mathbf{x} = \mathbf{b} $:

$$\overbrace{\begin{bmatrix}
a_{00} & a_{01}\\
a_{10} & a_{11}
\end{bmatrix}}^{\mathbf{A}}
\overbrace{
\begin{bmatrix}
x_0\\
x_1
\end{bmatrix}}
^{\mathbf{x}}
= \overbrace{\begin{bmatrix}
b_0\\
b_1
\end{bmatrix}}^{\mathbf{b}},
$$
where
$ \mathbf{A} \mathbf{x}$ is the *matrix multiplication*

$$ \mathbf{A} \mathbf{x} = \begin{bmatrix}
a_{00} x_0 + a_{01} x_1\\
a_{10} x_0 + a_{11} x_1
\end{bmatrix}.
$$

We say that $\mathbf{A}$ is a [*matrix*](https://en.wikipedia.org/wiki/Matrix_(mathematics)) and its dimension/shape is $2$-by-$2$:
- The first dimension/axis has size $2$. We also say that the matrix has $2$ rows.
- The second dimension/axis has size $2$. We also say that the matrix has $2$ columns.
$\mathbf{x}$ and $\mathbf{b}$ are called column vectors, which are matrices with one column.

Consider the example

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1,
\end{aligned}$$

or in matrix form with

$$ \begin{aligned}
\mathbf{A}&=\begin{bmatrix}
a_{00} & a_{01} \\
a_{10} & a_{11} 
\end{bmatrix} 
= \begin{bmatrix}
2 & 2 \\
0 & 2 
\end{bmatrix}\\
\mathbf{b}&=\begin{bmatrix}
b_0\\
b_1
\end{bmatrix} = \begin{bmatrix}
1\\
1
\end{bmatrix}\end{aligned}$$

Instead of using `list` to store the matrix, we will use a `numpy` array.

In [9]:
A = np.array([[2.0, 2], [0, 2]])
b = np.array([1.0, 1])
A, b

(array([[2., 2.],
        [0., 2.]]),
 array([1., 1.]))

Compared to `list`, `numpy` array is often more efficient and has more attributes.

In [10]:
array_attributes = set(attr for attr in dir(np.array([])) if attr[0] != "_")
list_attributes = set(attr for attr in dir(list) if attr[0] != "_")
print("\nCommon attributes:\n", *(array_attributes & list_attributes))
print("\nArray-specific attributes:\n", *(array_attributes - list_attributes))
print("\nList-specific attributes:\n", *(list_attributes - array_attributes))


Common attributes:
 copy sort

Array-specific attributes:
 dumps ndim strides tofile nbytes flags base resize tolist newbyteorder argmin dot setfield mean clip byteswap max cumsum reshape conj ctypes setflags any var T dump getfield tostring tobytes dtype searchsorted argpartition argmax min repeat itemset shape ptp astype transpose ravel flatten flat cumprod sum argsort diagonal all std choose item conjugate partition put compress prod round nonzero itemsize trace take data swapaxes fill size imag squeeze view real

List-specific attributes:
 clear pop count index reverse remove extend append insert


The following attributes give the dimension/shape, number of dimensions, size, and data type.

In [11]:
for array in A, b:
    print(
        f"""{array}
    shape: {array.shape}
    ndim: {array.ndim}
    size: {array.size}
    dtype: {array.dtype}
    """
    )

[[2. 2.]
 [0. 2.]]
    shape: (2, 2)
    ndim: 2
    size: 4
    dtype: float64
    
[1. 1.]
    shape: (2,)
    ndim: 1
    size: 2
    dtype: float64
    


Note that the function `len` only returns the size of the first dimension:

In [12]:
assert A.shape[0] == len(A)
len(A)

2

Unlike `list`, every `numpy` array has a data type. For efficient computation/storage, numpy implements different data types with different storage sizes:
* integer: `int8`, `int16`, `int32`, `int64`, `uint8`, ...
* float: `float16`, `float32`, `float64`, ...
* complex: `complex64`, `complex128`, ...
* boolean: `bool8`
* Unicode: `string`
* Object: `object`

E.g., `int64` is the 64-bit integer. Unlike `int`, `int64` has a range.

In [13]:
%%optlite -h 400
import numpy as np
min_int64 = np.int64(-2**63)
max_int64 = np.int64(2**63-1)
np.int64(2 ** 63)  # overflow error

OPTWidget(value=None, height=400, script='import numpy as np\nmin_int64 = np.int64(-2**63)\nmax_int64 = np.int…

We can use the `astype` method to convert the data type:

In [14]:
A_int64 = A.astype(int)  # converts to int64 by default
A_float32 = A.astype(np.float32)  # converts to float32
for array in A_int64, A_float32:
    print(array, array.dtype)

[[2 2]
 [0 2]] int64
[[2. 2.]
 [0. 2.]] float32


We must be careful about assigning items of different types to an array.

In [15]:
A_int64[0, 0] = 1
print(A_int64)
A_int64[0, 0] = 0.5
print(A_int64)  # intended assignment fails
np.array([int(1), float(1)])  # will be all floating point numbers

[[1 2]
 [0 2]]
[[0 2]
 [0 2]]


array([1., 1.])

**Exercise** Create a heterogeneous `numpy` array to store both integers and strings:
```Python
[0, 1, 2, 'a', 'b', 'c']
```
```{hint}
There is an `numpy` data type called `object`.
```

In [33]:
np.object?
heterogeneous_np_array = np.array([0, 1, 2, 'a', 'b', 'c'], dtype=np.object)
heterogeneous_np_array

/tmp/ipykernel_3486/639455601.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  heterogeneous_np_array = np.array([0, 1, 2, 'a', 'b', 'c'], dtype=np.object)


array([0, 1, 2, 'a', 'b', 'c'], dtype=object)

Init signature: np.object()
Docstring:     
The base class of the class hierarchy.

When called, it accepts no arguments and returns a new featureless
instance that has no instance attributes and cannot be given any.
Type:           type
Subclasses:     type, async_generator, int, bytearray_iterator, bytearray, bytes_iterator, bytes, builtin_function_or_method, callable_iterator, PyCapsule, ...


Be careful when creating arrays of `tuple`/`list`:

In [63]:
# %%optlite -h 350
# import numpy as np
a1 = np.array([(1, 2), [3, 4, 5]], dtype=object)  # [(1, 2) list([3, 4, 5])]
print(a1.shape, a1.size)  # (2,) 2
a2 = np.array([(1, 2), [3, 4]], dtype=object)  # [[1 2] \ [3 4]]
print(a2.shape, a2.size)  # (2, 2) 4

(2,) 2
(2, 2) 4


`numpy` provides many functions to create an array:

In [38]:
np.zeros?

Docstring:
zeros(shape, dtype=float, order='C', *, like=None)

Return a new array of given shape and type, filled with zeros.

Parameters
----------
shape : int or tuple of ints
    Shape of the new array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    The desired data-type for the array, e.g., `numpy.int8`.  Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: 'C'
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    NumPy arrays. If an array-like passed in as ``like`` supports
    the ``__array_function__`` protocol, the result will be defined
    by it. In this case, it ensures the creation of an array object
    compatible with that passed in via this argument.

    .. versionadded:: 1.20.0

Returns
-------
out : ndarray
    Array of zeros with the given shape, dtype, and order.

See Al

In [50]:
# %%optlite -h 400 -l
import numpy as np
a = [np.zeros(0)]  # shape = 0
a.append(np.zeros(1, dtype=int))  # shape = 1
a.append(np.zeros((2, 3, 4)))  # shape = (2, 3, 4)
a

[array([], dtype=float64),
 array([0]),
 array([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])]

In [ ]:
np.empty?

In [51]:
# %%optlite -h 400 -l
import numpy as np
a = [np.empty(2)]  # uninitalized
a.append(np.empty((2, 3, 4), 
                  dtype=int))
a

[array([1.39067116e-309, 8.28904606e-317]),
 array([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]])]

In [ ]:
np.ones?

In [52]:
# %%optlite -h 400 -l
import numpy as np
a = [np.ones(2)]
a.append(np.ones((2, 3, 4),
                dtype=int))
a

[array([1., 1.]),
 array([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],
 
        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]])]

In [ ]:
np.eye?

In [53]:
# %%optlite -l -h 400
import numpy as np
a = [np.eye(2)]  # dtype=float
a.append(np.eye(3, dtype=int))  # unit matrix
a

[array([[1., 0.],
        [0., 1.]]),
 array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])]

In [48]:
np.diag?

Signature: np.diag(v, k=0)
Docstring:
Extract a diagonal or construct a diagonal array.

See the more detailed documentation for ``numpy.diagonal`` if you use this
function to extract a diagonal and wish to write to the resulting array;
whether it returns a copy or a view depends on what version of numpy you
are using.

Parameters
----------
v : array_like
    If `v` is a 2-D array, return a copy of its `k`-th diagonal.
    If `v` is a 1-D array, return a 2-D array with `v` on the `k`-th
    diagonal.
k : int, optional
    Diagonal in question. The default is 0. Use `k>0` for diagonals
    above the main diagonal, and `k<0` for diagonals below the main
    diagonal.

Returns
-------
out : ndarray
    The extracted diagonal or constructed diagonal array.

See Also
--------
diagonal : Return specified diagonals.
diagflat : Create a 2-D array with the flattened input as a diagonal.
trace : Sum along diagonals.
triu : Upper triangle of an array.
tril : Lower triangle of an array.

Examples

In [55]:
# %%optlite -l -h 400
import numpy as np
a = [np.diag(range(1))]
a.append(np.diag(range(2)))
a.append(np.diag(np.ones(3), k=1))
# 1 above main diagonal
x = np.arange(9).reshape((3,3))
a.append(np.diag(x))
a.append(np.diag(np.diag(x)))
a

[array([[0]]),
 array([[0, 0],
        [0, 1]]),
 array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]]),
 array([0, 4, 8]),
 array([[0, 0, 0],
        [0, 4, 0],
        [0, 0, 8]])]

`numpy` also provides functions to build an array using rules.

In [ ]:
np.arange?

In [56]:
# %%optlite -l -h 400
# import numpy as np
a = [np.arange(5)]
# like range but allow non-integer parameters
a.append(np.arange(2.5, 5, 0.5))
a

[array([0, 1, 2, 3, 4]), array([2.5, 3. , 3.5, 4. , 4.5])]

In [ ]:
np.linspace?

In [59]:
# %%optlite -l -h 400
# import numpy as np
# can specify the number of points instead of the step size
a = [np.linspace(0, 4, 5)]
a.append(np.linspace(2.5, 4.5, 5))
a.append(np.linspace(0, 10, 4))
a

[array([0., 1., 2., 3., 4.]),
 array([2.5, 3. , 3.5, 4. , 4.5]),
 array([ 0.        ,  3.33333333,  6.66666667, 10.        ])]

In [ ]:
np.fromfunction?

In [62]:
# %%optlite -l -h 400
# import numpy as np
a = np.fromfunction(lambda i, j: (i + 1) * (j + 1), (3, 4))
a

array([[ 1.,  2.,  3.,  4.],
       [ 2.,  4.,  6.,  8.],
       [ 3.,  6.,  9., 12.]])

We can also reshape an array using the `reshape` method/function:

In [ ]:
np.ndarray.reshape?

In [70]:
# %%optlite -l -h 500
# import numpy as np
a = np.arange(2 * 3 * 4)
c1 = a.reshape(2, 3, 4)
# automatically calculate the size specified as `-1`
c2 = a.reshape(2, 3, -1)
c3 = a.reshape(-1, 4, 3)
c1, c2, c3, a.reshape(2, 2, 2, 3)

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],
 
        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]]]),
 array([[[[ 0,  1,  2],
          [ 3,  4,  5]],
 
         [[ 6,  7,  8],
          [ 9, 10, 11]]],
 
 
        [[[12, 13, 14],
          [15, 16, 17]],
 
         [[18, 19, 20],
          [21, 22, 23]]]]))

In [71]:
# %%optlite -l -h 500
# import numpy as np
a = np.arange(2 * 3 * 4)
# default C ordering
c = a.reshape((2, 3, -1), order="C")
# F ordering
f = a.reshape((2, 3, -1), order="F")
c, f

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[[ 0,  6, 12, 18],
         [ 2,  8, 14, 20],
         [ 4, 10, 16, 22]],
 
        [[ 1,  7, 13, 19],
         [ 3,  9, 15, 21],
         [ 5, 11, 17, 23]]]))

The C ordering and F ordering can be implemented using list comprehension:

In [72]:
# %%optlite -l -h 500
# import numpy as np
a = np.arange(2 * 3 * 4)
# last axis index changes fastest
c = np.array(
    [[[a[i*3*4+j*4+k] 
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])
# first axis index changes fastest
f = np.array(
    [[[a[i+j*2+k*3*2]
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])
c, f

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[[ 0,  6, 12, 18],
         [ 2,  8, 14, 20],
         [ 4, 10, 16, 22]],
 
        [[ 1,  7, 13, 19],
         [ 3,  9, 15, 21],
         [ 5, 11, 17, 23]]]))

`ravel` is a particular case of reshaping an array to one dimension.  
A similar function `flatten` returns a copy of the array but `ravel` and `reshape` returns a dynamic view whenever possible.

In [73]:
array = np.arange(2 * 3 * 4).reshape(2, 3, 4)
array, array.ravel(), array.reshape(-1), array.ravel(order="F")
# 'C': 3rd dimension change fastest, then 2nd dimension, last 1st dimension
# 'F': 1st dimension change fastest, then 2nd dimension, last 3rd dimension

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0, 12,  4, 16,  8, 20,  1, 13,  5, 17,  9, 21,  2, 14,  6, 18, 10,
        22,  3, 15,  7, 19, 11, 23]))

**Exercise** Correct the following function to print every element of an array line-by-line.
```Python
def print_array_entries_line_by_line(array):
    for i in array:
        print(i)
```

In [77]:
def print_array_entries_line_by_line(array):
    for i in array:
        for j in i:
            print(j, end = ' ')
        print()


print_array_entries_line_by_line(np.arange(2 * 3 * 4).reshape(2, 3, 4))

[0 1 2 3] [4 5 6 7] [ 8  9 10 11] 
[12 13 14 15] [16 17 18 19] [20 21 22 23] 


### Operating on `numpy` arrays

**How to verify the solution of a system of linear equations?**

Before solving the system of linear equations, let us try to verify a solution to the equations:

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1
\end{aligned}
$$

`numpy` provides the function `matmul` and the operator `@` for matrix multiplication.

In [ ]:
print(np.matmul(A, np.array([0, 0])) == b)
print(A @ np.array([0, 0.5]) == b)

Note that the comparison on `numpy` arrays returns a boolean array instead of a boolean value, unlike the comparison operations on lists.

To check whether all items are true, we use the `all` method.

In [ ]:
print((np.matmul(A, np.array([0, 0])) == b).all())
print((A @ np.array([0, 0.5]) == b).all())

**How to concatenate arrays?**

We will operate on an augmented matrix of the coefficients:

$$ \begin{aligned} \mathbf{C} &= \begin{bmatrix}
\mathbf{A} & \mathbf{b}
\end{bmatrix}\\
&= \begin{bmatrix}
a_{00} & a_{01} & b_0 \\
a_{10} & a_{11} & b_1
\end{bmatrix} 
\end{aligned}
$$

`numpy` provides functions to create block matrices:

In [ ]:
np.block?
C = np.block([A, b.reshape(-1, 1)])  # reshape to ensure same ndim
C

To stack an array along different axes:

In [ ]:
array = np.arange(1 * 2 * 3).reshape(1, 2, 3)
for concat_array in [
    array,
    np.hstack((array, array)),  # stack along the first axis
    np.vstack((array, array)),  # second axis
    np.concatenate((array, array), axis=-1),  # last axis
    np.stack((array, array), axis=0),
]:  # new axis
    print(concat_array, "\nshape:", concat_array.shape)

**How to perform arithmetic operations on a `numpy` array?**

To divide all the coefficients by $2$, we can write:

In [ ]:
D = C / 2
D

Note that the above does not work for `list`.

In [ ]:
%%optlite -l -h 500
[[2, 2, 1], [0, 2, 1]] / 2

Arithmetic operations on `numpy` arrays apply if the arrays have compatible dimensions. Two dimensions are compatible when
- they are equal, except for
- components equal to 1.

`numpy` uses [broadcasting rules](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules) to stretch the axis of size 1 up to match the corresponding axis in other arrays.  
`C / 2` is an example where the second operand $2$ is broadcasted to a $2$-by-$2$ matrix before the elementwise division. Another example is as follows.

In [ ]:
three_by_one = np.arange(3).reshape(3, 1)
one_by_four = np.arange(4).reshape(1, 4)
print(
    f"""
{three_by_one}
*
{one_by_four}
==
{three_by_one * one_by_four}
"""
)

Next, to subtract the second row of the coefficients from the first row:

In [ ]:
D[0, :] = D[0, :] - D[1, :]
D

Notice the use of commas to index different dimensions instead of using multiple brackets:

In [ ]:
assert (D[0][:] == D[0, :]).all()

Using this indexing technique, it is easy to extract the last column as the solution to the system of linear equations:

In [ ]:
x = D[:, -1]
x

This gives the desired solution $x_0=0$ and $x_1=0.5$ for

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1\\
\end{aligned}$$

`numpy` provides many [convenient ways](https://numpy.org/doc/stable/reference/arrays.indexing.html#advanced-indexing) to index an array.

In [ ]:
B = np.arange(2 * 3).reshape(2, 3)
B, B[(0, 1), (2, 0)]  # selecting the corners using integer array

In [ ]:
B = np.arange(2 * 3 * 4).reshape(2, 3, 4)
B, B[0], B[0, (1, 2)], B[0, (1, 2), (2, 3)], B[
    :, (1, 2), (2, 3)
]  # pay attention to the last two cases

In [ ]:
assert (B[..., -1] == B[:, :, -1]).all()
B[..., -1]  # ... expands to selecting all elements of all previous dimensions

In [ ]:
B[B > 5]  # indexing using boolean array

Finally, the following function solves 2 linear equations with 2 variables.

In [ ]:
def solve_2_by_2_system(A, b):
    """Returns the unique solution of the linear system, if exists,
    else returns None."""
    C = np.hstack((A, b.reshape(-1, 1)))
    if C[0, 0] == 0:
        C = C[(1, 0), :]
    if C[0, 0] == 0:
        return None
    C[0, :] = C[0, :] / C[0, 0]
    C[1, :] = C[1, :] - C[0, :] * C[1, 0]
    if C[1, 1] == 0:
        return None
    C[1, :] = C[1, :] / C[1, 1]
    C[0, :] = C[0, :] - C[1, :] * C[0, 1]
    return C[:, -1]

In [ ]:
# tests
for A in (
    np.eye(2),
    np.ones((2, 2)),
    np.stack((np.ones(2), np.zeros(2))),
    np.stack((np.ones(2), np.zeros(2)), axis=1),
):
    print(f"A={A}\nb={b}\nx={solve_2_by_2_system(A,b)}\n")

### Universal functions

Why does the first line of code below return two arrays but the second code return only one array? Shouldn't the first line of code return the following?
```Python
array([[(0,1), (0,2), (0,3)],
       [(1,1), (1,2), (1,3)]])
```

In [ ]:
print(np.fromfunction(lambda i, j: (i, j), (2, 3), dtype=int))
print(np.fromfunction(lambda i, j: (i * j), (2, 3), dtype=int))

From the documentation, `fromfunction` applies the given function to the two arrays as arguments.
- The first line of code returns a tuple of the arrays.
- The second line of code multiplies the two arrays to give one array, according to how multiplication works for numpy arrays.

Indeed, `numpy` implements [universal/vectorized functions/operators](https://numpy.org/doc/stable/reference/ufuncs.html) that take arrays as arguments and perform operations with appropriate broadcasting rules. The following is an example that uses the universal function `np.sin`:

In [ ]:
import matplotlib.pyplot as plt


@widgets.interact(a=(0, 5, 1), b=(-1, 1, 0.1))
def plot_sin(a=1, b=0):
    x = np.linspace(0, 2 * math.pi)
    plt.plot(x, np.sin(a * x + b * math.pi))  # np.sin, *, + are universal functions
    plt.title(r"$\sin(ax+b\pi)$")
    plt.xlabel(r"$x$ (radian)")

In addition to making the code shorter, universal functions are both efficient and flexible. (Recall the Monte Carlo simulation to approximate $\pi$.)

**Exercise** Explain how the Monte Carlo simulation work using universal functions:
```Python
def np_approximate_pi(n):
    in_circle = (np.random.random((n,2))**2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n**0.5
    return np.array([mean - 2*std, mean + 2*std])
```

YOUR ANSWER HERE